### Widgets don't just have traits, they `HaveTraits`

In [ ]:
slider.__class__

ipywidgets.widgets.widget_int.IntSlider

In [ ]:
import inspect

inspect.getmro(slider.__class__)

(ipywidgets.widgets.widget_int.IntSlider,
 ipywidgets.widgets.widget_int._BoundedInt,
 ipywidgets.widgets.widget_int._Int,
 ipywidgets.widgets.widget_description.DescriptionWidget,
 ipywidgets.widgets.domwidget.DOMWidget,
 ipywidgets.widgets.valuewidget.ValueWidget,
 ipywidgets.widgets.widget_core.CoreWidget,
 ipywidgets.widgets.widget.Widget,
 ipywidgets.widgets.widget.LoggingHasTraits,
 traitlets.traitlets.HasTraits,
 traitlets.traitlets.HasDescriptors,
 object)

As we can see, the slider inherits `traitlets.traitlets.HasTraits`. 

You can get a list of traits belonging to an object that `HasTraits` by looking at the trait called `keys`.


In [ ]:
slider.keys

['_dom_classes',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'behavior',
 'continuous_update',
 'description',
 'description_allow_html',
 'disabled',
 'layout',
 'max',
 'min',
 'orientation',
 'readout',
 'readout_format',
 'step',
 'style',
 'tabbable',
 'tooltip',
 'value']

Now we know that our IntSlider has three traits we are really interested in: min, max, and value. They all seem to return `int`s.

In [ ]:
(slider.value, slider.min, slider.max)

(46, 0, 100)

Let's start by considering `min`, `max`, and `value`. Maybe we want to figure out what *type* of trait these are. Traits come in many types including `Int`, `Float`, `Unicode` (string), `List`, and `Any`, which allows for any kind of object. Since our slider is of type `IntSlider`, we might imagine that the trait type is also type `traitlets.traitlets.Int`. When we display the value of slider, that seems pretty reasonable...

But that doesn't actually tell us the type of the trait object. To get that information, we need to use the `traits()` method.

In [ ]:
slider.traits()

{'_dom_classes': <ipywidgets.widgets.trait_types.TypedTuple>,
 '_model_module': <traitlets.traitlets.Unicode>,
 '_model_module_version': <traitlets.traitlets.Unicode>,
 '_model_name': <traitlets.traitlets.Unicode>,
 '_msg_callbacks': <traitlets.traitlets.Instance>,
 '_property_lock': <traitlets.traitlets.Dict>,
 '_states_to_send': <traitlets.traitlets.Set>,
 '_view_count': <traitlets.traitlets.Int>,
 '_view_module': <traitlets.traitlets.Unicode>,
 '_view_module_version': <traitlets.traitlets.Unicode>,
 '_view_name': <traitlets.traitlets.Unicode>,
 'behavior': <traitlets.traitlets.CaselessStrEnum>,
 'comm': <traitlets.traitlets.Any>,
 'continuous_update': <traitlets.traitlets.Bool>,
 'description': <traitlets.traitlets.Unicode>,
 'description_allow_html': <traitlets.traitlets.Bool>,
 'disabled': <traitlets.traitlets.Bool>,
 'keys': <traitlets.traitlets.List>,
 'layout': <ipywidgets.widgets.trait_types.InstanceDict>,
 'log': <traitlets.traitlets.Instance>,
 'max': <traitlets.traitlets.CI

Okay, this is a lot to digest. This is a dictionary. The keys are the names of the traits, and the values show us the trait type. Let's take a closer look...
```python
 'max': <traitlets.traitlets.CInt at 0x7ff8efdd9580>,
 'min': <traitlets.traitlets.CInt at 0x7ff8efdd95e0>,
 'value': <traitlets.traitlets.CInt at 0x7ff8efdd94c0>
```
We can see that these traits are all of type `CInt`. What does that mean? How is that different from `Int`? It all has to do with the validation methods that ship with the traits.

In [ ]:
from traitlets import HasTraits, Int

class MyInt(HasTraits):
    
    value = Int()
    
    def __init__(self):
        pass

In [ ]:
myint = MyInt()
myint.value

0

Okay, we expect the `value` trait to only accept integers right? Let's find out by giving it a float instead.

In [ ]:
myint.value = 2.5

TraitError: The 'value' trait of a MyInt instance expected an int, not the float 2.5.

Okay! As expected, our trait of type `Int` doesn't take floats. We can try strings too.

In [ ]:
myint.value = '2'

TraitError: The 'value' trait of a MyInt instance expected an int, not the str '2'.

In [ ]:
That looks like an `Int`

In [ ]:
(slider.min, slider.max, slider.value)

(0, 100, 0)

### Under the hood: traitlets

So how do ipywidgets work under the hood exactly? Well, ipywidgets are built on top of the [traitlets](https://traitlets.readthedocs.io/en/stable/using_traitlets.html#) library. Widgets inherit `HasTraits`, which gives us the ability to observe changes in its value. Let's build an object that works like a widget, but doesn't actually have a view. 

In [ ]:
from traitlets import HasTraits, Int

class Number(HasTraits):
    
    value = Int()
    square = Int()
    
    def __init__(self):
        pass # what goes here?
        # how do we observe the change
        
    def on_value_change(self, change):
        self.square = change['new']**2

In [ ]:
number = Number()
number.value = 2
number.square